In [1]:
import pandas as pd

In [2]:
# Download EN wikipedia locally.

# import requests
# import io

# WIKIPEDIA = ["https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/0.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/1.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/2.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/3.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/4.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/5.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/6.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/7.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/8.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/9.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/10.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/11.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/12.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/13.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/14.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/15.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/16.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/17.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/18.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/19.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/20.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/21.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/22.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/23.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/24.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/25.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/26.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/27.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/28.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/29.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/30.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/31.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/32.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/33.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/34.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/35.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/36.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/37.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/38.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/39.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/40.parquet"]

# for i, url in enumerate(WIKIPEDIA):
#     print(url)
#     pd.read_parquet(io.BytesIO(requests.get(url).content)).rename(columns={'id': 'article_id'}).to_parquet(f"wiki/wiki_{i}.parquet")

In [3]:
import pandas as pd

def select_top_counts():
    selected = {}
    with open('../transformed/data/links.txt', 'r', encoding='utf-8', errors='replace') as file:
        for line in file:
            try:
                # Filter out hashtag pages
                if line[0] == '#':
                    continue

                # Filter out category pages
                skip = False
                for substring in ['Category:', 'File:', 'Wikipedia:', 'user:']:
                    if substring in line:
                        skip = True
                if skip:
                    continue
                    
                # Filter out aliases and sublinks
                for substring in ['#', '|']:
                    if substring in line:
                        line = line.split(substring, 1)[0]

                line = line.strip()

                if line in selected:
                    selected[line] += 1
                else:
                    selected[line] = 1
            except Exception as e:
                print(e)

    selected = pd.DataFrame(
        list(
            filter(
                lambda x: x['count'] >= 35,
                sorted(
                    [
                        {
                            'lower_title': x.lower().strip(), 
                            'count': y
                        } 
                        for x, y in selected.items()
                    ],
                    key=lambda x: x['count'],
                    reverse=True
                )   
            )
        )
    )

    return selected

selected = select_top_counts()
selected.__len__()

79406

In [4]:
import os
import pandas as pd
from typing import Set 

def filter_df(df, selected):
    df = df.rename(columns={'id': 'article_id'})
    df['lower_title'] = df.title.apply(lambda x: x.strip().lower())
    return df.merge(selected, on='lower_title')

In [5]:
first = True
for file in os.listdir('wiki'):
    print(file)
    if first:
        df = filter_df(pd.read_parquet(f'wiki/{file}'), selected)
        first = False
        continue
    df = pd.concat(
        [
            df, 
            filter_df(pd.read_parquet(f'wiki/{file}'), selected)
        ], 
        axis=0
    )
df = df.drop_duplicates(['article_id'])

wiki_0.parquet
wiki_1.parquet
wiki_10.parquet
wiki_11.parquet
wiki_12.parquet
wiki_13.parquet
wiki_14.parquet
wiki_15.parquet
wiki_16.parquet
wiki_17.parquet
wiki_18.parquet
wiki_19.parquet
wiki_2.parquet
wiki_20.parquet
wiki_21.parquet
wiki_22.parquet
wiki_23.parquet
wiki_24.parquet
wiki_25.parquet
wiki_26.parquet
wiki_27.parquet
wiki_28.parquet
wiki_29.parquet
wiki_3.parquet
wiki_30.parquet
wiki_31.parquet
wiki_32.parquet
wiki_33.parquet
wiki_34.parquet
wiki_35.parquet
wiki_36.parquet
wiki_37.parquet
wiki_38.parquet
wiki_39.parquet
wiki_4.parquet
wiki_40.parquet
wiki_5.parquet
wiki_6.parquet
wiki_7.parquet
wiki_8.parquet
wiki_9.parquet


In [6]:
df.drop('text', axis=1).to_parquet('data/articles_over_35.parquet')

In [7]:
chunks = [] 
i = 0
stride = 25

for _, row in df.iterrows():
    start = 0
    end = 50
    text = row['text'].split()
    while end < len(text):
        chunks.append({
            'article_id': row['article_id'], 
            'chunk_id': i, 
            'chunk': ' '.join(text[start : end])
        })
        i += 1
        
        start += stride 
        end += stride
    
    if start <= len(text):
        chunks.append({
            'article_id': row['article_id'], 
            'chunk_id': i, 
            'chunk': ' '.join(text[start :])
        })
        i += 1

del(df)
chunks = pd.DataFrame(chunks)
len(chunks)

7671920

In [8]:
chunks.to_parquet('data/chunks_over_35.parquet')

In [20]:
def print_text(text):
    start = 0
    end = 100
    while end < len(text):
        print(text[start : end])
        start += 100
        end += 100
    print(text[end :])

print_text(chunks.sample().iloc[0]['chunk'])

included Chattanooga, Tennessee natives Sam Gooden, Richard Brooks, and his brother Arthur Brooks. B
y 1958, the Roosters had a new manager in Eddie Thomas, a record deal with Vee-Jay Records, and a ne
w name: Jerry Butler & the Impressions. The group's first hit single was 1958's "For Your Precious L
ove", which hit No. 11 on the US pop chart and No. 3 on the R&B chart. However, soon after the relea

